#2025-03-30 FELIPE GARCÍA: MODELO SIRD + ML

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from scipy.optimize import curve_fit

# Cargar los datos
data = pd.read_csv(r"C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\data\Evolución_enfermedad_fiebre_de_Norilsk.csv")

# Extraer columnas relevantes
susceptibles = data['Susceptibles'].values
infectados = data['Infectados'].values
muertos = data['Muertos'].values
tiempo = np.arange(len(data))

# Estimar removidos (muertos asumidos como removidos)
removidos = np.cumsum(muertos)

# Definir el modelo SIRD
def sird_model(y, t, beta, gamma, mu):
    S, I, R = y
    dSdt = -beta * S * I / (S + I + R)
    dIdt = beta * S * I / (S + I + R) - gamma * I - mu * I
    dRdt = gamma * I + mu * I
    return [dSdt, dIdt, dRdt]

# Función de ajuste de parámetros
def fit_sird(t, beta, gamma, mu):
    sol = odeint(sird_model, [S0, I0, R0], t, args=(beta, gamma, mu))
    return sol[:, 1]  # Ajustar a los datos de infectados

# Condiciones iniciales
S0 = susceptibles[0]
I0 = infectados[0]
R0 = removidos[0]

# Ajuste de parámetros con curve_fit
params, _ = curve_fit(fit_sird, tiempo, infectados, p0=[0.4, 0.1, 0.05], bounds=(0, [1, 1, 1]))
beta_opt, gamma_opt, mu_opt = params

# Resolver la ODE con los parámetros ajustados
sol = odeint(sird_model, [S0, I0, R0], tiempo, args=(beta_opt, gamma_opt, mu_opt))
S_pred, I_pred, R_pred = sol.T

# Gráfica de evolución
plt.figure(figsize=(10,6))
plt.plot(tiempo, S_pred, label='Susceptibles (Predicción)')
plt.plot(tiempo, I_pred, label='Infectados (Predicción)')
plt.plot(tiempo, R_pred, label='Removidos (Predicción)')
plt.scatter(tiempo, susceptibles, label='Datos Susceptibles', color='blue', alpha=0.5)
plt.scatter(tiempo, infectados, label='Datos Infectados', color='red', alpha=0.5)
plt.scatter(tiempo, removidos, label='Datos Removidos', color='green', alpha=0.5)
plt.legend()
plt.xlabel('Tiempo (semanas)')
plt.ylabel('Población')
plt.title('Modelo SIRD - Proyección Epidemia con Parámetros Ajustados')
plt.show()

# Implementación de ML para mejorar la predicción con Random Forest
X = np.column_stack([susceptibles, infectados, removidos])
y = np.roll(infectados, -1)  # Predecir infectados en el siguiente paso
model_rf = RandomForestRegressor(n_estimators=100)
model_rf.fit(X[:-1], y[:-1])

# Predicción con Random Forest
pred_rf = model_rf.predict(X)

# Implementación de LSTM para series temporales
X_lstm = np.reshape(X, (X.shape[0], X.shape[1], 1))
y_lstm = y
model_lstm = Sequential([
    LSTM(50, activation='relu', input_shape=(X.shape[1], 1)),
    Dense(1)
])
model_lstm.compile(optimizer=Adam(), loss='mse')
model_lstm.fit(X_lstm[:-1], y_lstm[:-1], epochs=50, verbose=0)

# Predicción con LSTM
pred_lstm = model_lstm.predict(X_lstm)

# Graficar comparaciones
plt.figure(figsize=(10,6))
plt.plot(tiempo, infectados, label='Infectados Reales', color='red')
plt.plot(tiempo, pred_rf, label='Predicción Random Forest', linestyle='dashed')
plt.plot(tiempo, pred_lstm, label='Predicción LSTM', linestyle='dotted')
plt.xlabel('Tiempo (semanas)')
plt.ylabel('Número de Infectados')
plt.title('Comparación de Modelos de Machine Learning')
plt.legend()
plt.show()

KeyError: 'Susceptibles'